# [Lesson 1](https://www.youtube.com/watch?v=8SF_h3xF3cE&t=414s) - Deep Learning for Coders (Jeremy Howard)

**Idea:** primeros pasos en Deep Learning. Con pocas lineas de codigo podemos crear un modelo de computer vision que sea un clasificador de aves

**También se habla de:**
- dall-e que a partir de una descripción (en texto), con Deep Learning, genera nuevas imágenes.
- no todo es sobre imagenes! hay un modelo de lenguaje llamado Pathways (PaLM Pathways Language Model) de Google, que puede tomar cualquier texto arbitrario en inglés, una pregunta y puede crear una respuesta que no sólo responde la pregunta, sino que tabien explica su pensamiento (2022). Por ejemplo, puede explicar un chiste.. Text : input -- Model --\ Text : output
- data ethics

**Otros:**
- Metodologia cups (3 colores, 3 estados respecto a lo que estoy aprendiendo)
- [Learn Machine Learning](https://rosmulski.gumroad.com/l/learn_machine_learning) meta learning: how to learn deep learning and thrive in the digital world

**Sobre Jeremy:**
- Jeremy es el inventor del algoritmo ULMFiT que segun el libro Transformers fue uno de los dos fundamentos clave detras de la revolucion de la NLP (Natural Language Processing with Transformers, Building Language Applications with Hugging Face)
- [Howard and Ruder (2018)](https://www.researchgate.net/publication/322592284_Fine-tuned_Language_Models_for_Text_Classification) introduced Universal Language Model Fine-tuning (ULMFiT) ([PDF](https://github.com/sofiavalino/fastbook/pdfs/Fine-tuned_Language_Models_for_Text_Classification.pdf)) to facilitate transfer learning across NLP tasks. 
    
        ULMFiT includes three main procedures: (1) language model pre-training, (2) model fine-tuning (i.e., discriminative fine-tuning), and (3) classifier fine-tuning. Because the model has already captured the general properties of the language during pre-training, it is proposed that a relatively minor tweaking (i.e., fine-tuning) is sufficient for adapting the model to a specific task – e.g., text classification or generation. ULMFiT represents a strong contribution in NLP, paving the way for more advanced transfer learning models, such as those based on transformers (Wolf et al., 2019) – including GPT-2. Therefore, we consider ULMFiT as a solid baseline for our experiments.

**Vamos a usar Pytorch (mas popular que tensorflow en los ultimos años):**
- si bien es optimo por fundamentos tecnicos, requiere mucho codigo complejo para cosas relativamente simples
- base solida sobre la cual construir 
- La libreria [fastai](https://docs.fast.ai) fue construida sobre pytorch, donde se reduce demasiado el codigo y mantiene la solidez (menos codigo es mejor =D). Sin embargo puede no usarse mucho... leer en [Reddit](https://www.reddit.com/r/learnmachinelearning/comments/i0d6c5/why_do_most_people_not_use_fastai_library/?rdt=45629).

**Notebook Servers**
course.fast.ai ¿como usar servidores en la nube dif para ejecutar las notebooks?
https://forums.fast.ai/t/setting-up-notebook-servers/91203


https://youtu.be/8SF_h3xF3cE?t=2176

In [ ]:
from fastbook import *

In [ ]:
# Buscamos en DuckDuckGo imágenes de aves
urls = search_images_ddg('bird photos', max_images = 1)
len(urls), urls[0]

In [ ]:
# descargamos la imagen
dest = Path('bird.jpg')
if not dest.exists(): download_url(urls[0], dest, show_progress=False)

In [ ]:
# visualizamos la imagen
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
Todos los modelos necesitan inputs numéricos.
Las imagenes están hechas de números.
PixSpy: es una página que dada una imagen, permite visualizar cada pixel con su representación numérica entre 0 y 255 (red,green,blue)
 

In [ ]:
# necesitamos comparar con imagenes que no sean un pajaro
# por ejemplo, se toman imagenes de bosques
# buscamos y descargamos 200 fotos de bosques y pajaros
# luego se redimensionan para que no tengan mas de 400 pixeles
# no necesitamos que sea una imagen grande y lleva demasiado tiempo abrirla

searches = 'forest','bird'
path = Path('bird_or_not')

if not path.exists():
    for o in searches:
        dest = (path/o)
        dest.mkdir(exist_ok = True)
        results = search_images_ddg(f'{o} photo')
        download_images(dest, urls=results[:200])
        resize_images(dest, max_size=400, dest=dest)

In [ ]:
# cuando descargamos imagenes, a menudo estan rotas
# si queremos entrenar un modelo y usamos imagenes rotas, no funcionara
# usamos el siguiente metodo que "valida" las imagenes que están ok y las que no, las elimina
failed = verify_images(get_image_files(path))
failed.map(Path.unlink); # unlink/desvincular aquellas que no sirvan

In [ ]:
# ahora creamos un "datablock" o "bloque de datos"
# otorgandole a la libreria de fast.ai toda la info
# que requiere para crear un modelo de computer vision 
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files, # dame todas las imagenes que recolectamos
    splitter=Randomsplitter(valid_pct=0.2, seed=42),
    get_y=parend_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n = 6) #mostra 6

In [ ]:
# Algo muy interesante de los modelos de computer vision es que
# es muy facil verificar los datos porque pueden simplemente "mirarlos" (vision, je)
# lo que no es el caso de otros modelos

In [ ]:
# esto va a recorrer cada foto de las 400 (200 de bosques y 200 de aves)
# y para las que son de bosques, aprendera un poco mas sobre como se ve un bosque (idem parajos)

learn = cnn_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(3)

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"Thiss is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

## Parentesis para reflexionar 

### ¿Cómo es que hacemos un clasificador de aves en 2 min y antes no se podia? 

Menciona como se hacia el reconocimiento de imagenes en el 2013, hay un proyecto de stanford de reconocimiento de patologias de cancer de mama, es un enfoque clasico de machine learning. Se crearon miles y miles de features, llevo años y mucho codigo y matematica, luego de tener todas las features lo metieron en un modelo de regresion logistica.

La diferencia con las redes neuronales es que, **no necesitan** que creemos las features, sino que **las crean por nosotros**. No le damos features, sino que le pedimos que aprenda features.
Deep learning es **profundo**, porque podemos tomar las features y combinarlas, creando caracteristicas mas avanzadas. En cada CAPA se encontrara una mayor complejidad.

Con una red neuronal, que es la funcion basica utilizada en deep learning, no tenemos que codificar manualmente ninguna de las funciones ni idear ninguna de las ideas, solo comenzamos con una red neuronal aleatoria y le damos ejemplos, haciendo que aprenda a reconocerlas, creando cosas por si misma. Esto se puede hacer ahora porque no debemos codificar las features nosotros, sino que todo se puede aprender mediante redes neuronales.

### ¿Cómo construir algoritmos basados en imagenes?

No son solo para imagenes, en este curso sera un tema general. En este curso veremos tecnicas fundamentales, que con creatividad, estas tecnicas pueden utilizarse de manera muy amplia.

Algunos ejemplos de aplicacion de un clasificador de imagenes:

* A partir de sonidos crear imagenes y usar un clasificador de imagenes para clasificar los sonidos (niños gritando)
* Tomar series de tiempo, convertirlas en imagenes y clasificarlas
* Crear imagenes a partir de movimientos del cursor (clicks puntos y movimientos lineas, velocidad de movimiento se convirtio en colores)

### Transfer Learning
